In [74]:
#### IMPORTS #####
from collections import deque
from datetime import timedelta
import ujson
import pandas as pd
from datetime import datetime
import praw
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.svm import SVC
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import argparse
import gzip
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import time
import numpy as np

In [75]:
BANNED_LOC = "Data/banned_reddits.txt"
banned_dict = {}
with open(BANNED_LOC) as f:
    for l in f:
        banned_dict[l.strip()[2:]] = True

In [102]:
t0 = time.time()

# make a queue much faster than list for adding large amounts of items.
q = deque()

# Read in JSON data
NUM_POSTS_TO_READ = 1000000
MSG_PREV = 30

MESSAGES_LOC = "Data/RC_2019-01"

# header = ("%-23s | %-20s | %"+str(MSG_PREV+10)+"s")%("Subreddit", "User", "Message")
# print(header)
# print("-"*len(header))

with open(MESSAGES_LOC, "r") as f:
    num_read = 0
    for line in f:
        num_read += 1
        if num_read > NUM_POSTS_TO_READ: break
        post = ujson.loads(line)
        post["banned"] = True if post["subreddit"] in banned_dict else False
        q.append(post)
#         print(("/r/%-20s | %-20s | %"+str(MSG_PREV+10)+"s")%( post['subreddit'], post['author'], repr(post['body'][:MSG_PREV]))) #post['created_utc']

df = pd.DataFrame(q)
t1 = time.time()
print("took: %.2f"%(t1-t0))

took: 46.00


In [105]:
# print the dataframe
df[["subreddit", "author", "body", "banned"]]

tmp = df.iloc[np.where(df["banned"])]
for row in tmp[["subreddit","author", "body"]].values:
    print("%-25s|"%repr(row[0]), "%-25s|"%repr(row[1]), repr(row[2][:100]))

'CringeAnarchy'          | '[deleted]'              | '[deleted]'
'nbastreams'             | 'AutoModerator'          | '#[Join our Discord for LIVE game chat!](https://discord.gg/FphMn2F), Streams are NOT posted on Disco'
'nbastreams'             | '[deleted]'              | '[deleted]'
'nbastreams'             | 'youpit'                 | '**HD** Adaptive | [Minnesota Timberwolves @ New Orleans Pelicans](http://youpit.me/2019/01/01/nba-mi'
'nbastreams'             | 'AlphaSports'            | 'SD 520p | [English Stream](http://primealpha.pro/nba-6.php) | Ad Overlay: 3 | Mobile Compatible: No '
'nbastreams'             | '[deleted]'              | '[removed]'
'CringeAnarchy'          | 'B-L-G-Y'                | 'E M P O W E R M E N T'
'nbastreams'             | 'nba4live2019'           | '**HD** 720p | [LeaguePass AWAY | [Minnesota Timberwolves @ New Orleans Pelicans](http://nba4live.fun'
'nbastreams'             | 'AutoModerator'          | '#[Join our Discord for LIVE game chat!](h